# MCSM-Benchs: Using user-provided signals and performance metric

`MCSM-Benchs` can be helpful to create benchmarks with signals and performance metrics provided by the user.
This can be useful, for instance, when dealing with real-world signals and performance metrics that do not need a noiseless version of the signal.

In this notebook, we create a benchmark with real-world audio signals and a performance metric that is computed based on the original signal and the processed one.


In [1]:
import numpy as np
from numpy import pi as pi
import pandas as pd
from matplotlib import pyplot as plt
from mcsm_benchs.Benchmark import Benchmark
from mcsm_benchs.ResultsInterpreter import ResultsInterpreter
from mcsm_benchs.SignalBank import SignalBank
from utils import spectrogram_thresholding, get_stft

from IPython.display import Audio

## 1. Creating a dictionary of methods

Let's create a dictionary of methods to benchmark. As as example, we will compare two strategies for spectrogram thresholding.
The first one is hard thresholding, in which the thresholding function is defined as:
The second one is soft thresholding, here defined as:

These two approaches are implemented in the python function ```thresholding(signal, lam, fun='hard')``` function, which receives a signal to clean, a positional argument ```lam``` and a keyword argument ```fun``` that can be either ```hard``` or ```soft```.
 
Our dictionary of methods will consist then in two methods: hard thresholding and soft thresholding.
For both approaches, let's use a value of ```lam=1.0``` for now.

In [2]:

def method_1(noisy_signal, *args, **kwargs):
    # If additional input parameters are needed, they can be passed in a tuple using 
    # *args or **kwargs and then parsed.
    xr = spectrogram_thresholding(noisy_signal,1.0,fun='hard')
    return xr

def method_2(noisy_signal, *args, **kwargs):
    # If additional input parameters are needed, they can be passed in a tuple using 
    # *args or **kwargs and then parsed.
    xr = spectrogram_thresholding(noisy_signal,2.0,fun='soft') 
    return xr

# Create a dictionary of the methods to test.
my_methods = {
    'Hard_Thr': method_1, 
    'Soft_Thr': method_2,
    }

## 2. Creating a dictionary of user-provided signals

We load two synthesized speech signals, and fix the length to `N=2**13` samples.
With these signals, we create a dictionary, where the key is going to be used as an identifier of the signal in the benchmark final results.

In [3]:
# Loading signals and creating dictionary
N = 2**13
signals_dic = {
    'speech_1': np.loadtxt('6_female.csv')[0:N],
    'speech_2': np.loadtxt('6_male.csv')[0:N]
            }

In [4]:
# Listen to the signals
fs = 16000
Audio(signals_dic['speech_2'], rate=fs)

## 3. Defining a performance metric

We use the Perceptual Evaluation of Speech Quality (PESQ) metric as a performance metric.

To do this, we first create a wrapper `perf_fun(...)` of the function `pesq(...)`.
Performarmance metrics must follow the signature `perf_fun(x, xest, **kwargs)`, where
- `x` is the original signal (without added noise).
- `xest` is the output of a denoising approach.
- `**kwargs` is used to receive a number of extra parameters passed by the benchmark class when the function `perf_fun(...)` is called.

In [5]:
from pesq import pesq
# Create a wrapper function for PESQ.
# Normalize by the PESQ of the original signal.
perfuns = {'pesq1':lambda x,xest,**kwargs: pesq(fs,x,xest,'nb')/pesq(fs,x,x,'nb'),
           'pesq2':lambda x,xest,**kwargs: pesq(fs,x,xest,'nb'),}
perfuns['pesq1'](signals_dic['speech_1'],signals_dic['speech_1'])

1.0

Now we are ready to instantiate a `Benchmark` object and run a test using the proposed methods and parameters. The benchmark constructor receives a name of a task (which defines the performance function of the test), a dictionary of the methods to test, the desired length of the signals used in the simulation, a dictionary of different parameters that should be passed to the methods, an array with different values of SNR to test, and the number of repetitions that should be used for each test. Once the object is created, use the class method `run_test()` to start the experiments.

*Remark 1: You can use the ```verbosity``` parameter to show less or more messages during the progress of the experiments. There are 6 levels of verbosity, from ```verbosity=0``` (indicate just the start and the end of the experiments) to ```verbostiy = 5``` (show each method and parameter progress)*

*Remark 2: Parallelize the experiments is also possible by passing the parameter ```parallelize = True```. *

In [6]:
benchmark = Benchmark(task = 'denoising',
                        N = N,
                        methods = my_methods, 
                        SNRin = [0,10,20], 
                        repetitions = 10,
                        signal_ids=signals_dic, # Input user-defined signals
                        verbosity=5,
                        obj_fun=perfuns, # Define a performance metric
                        )
                        
benchmark.run_test() # Run the benchmark

Running benchmark...
- Signal speech_1
-- SNR: 0 dB
--- Method: Hard_Thr
---- Parameters Combination: 0
------ Inner loop. Hard_Thr: 0
------ Inner loop. Hard_Thr: 1
------ Inner loop. Hard_Thr: 2
------ Inner loop. Hard_Thr: 3
------ Inner loop. Hard_Thr: 4
------ Inner loop. Hard_Thr: 5
------ Inner loop. Hard_Thr: 6
------ Inner loop. Hard_Thr: 7
------ Inner loop. Hard_Thr: 8
------ Inner loop. Hard_Thr: 9
Elapsed:0.0034453153610229494
--- Method: Soft_Thr
---- Parameters Combination: 0
------ Inner loop. Soft_Thr: 0
------ Inner loop. Soft_Thr: 1
------ Inner loop. Soft_Thr: 2
------ Inner loop. Soft_Thr: 3
------ Inner loop. Soft_Thr: 4
------ Inner loop. Soft_Thr: 5
------ Inner loop. Soft_Thr: 6
------ Inner loop. Soft_Thr: 7
------ Inner loop. Soft_Thr: 8
------ Inner loop. Soft_Thr: 9
Elapsed:0.0036676406860351564
-- SNR: 10 dB
--- Method: Hard_Thr
---- Parameters Combination: 0
------ Inner loop. Hard_Thr: 0
------ Inner loop. Hard_Thr: 1
------ Inner loop. Hard_Thr: 2
-----

{'pesq1': {'speech_1': {0: {'Hard_Thr': {'((), {})': [0.2511395364342408,
      0.2461274473524261,
      0.24798206052382246,
      0.24903602886032414,
      0.25248074101920226,
      0.24812297925630133,
      0.2511082182483914,
      0.24970204480759678,
      0.2556525263266698,
      0.25082087716918405]},
    'Soft_Thr': {'((), {})': [0.38184138562766257,
      0.3922177951032719,
      0.3993604380918875,
      0.39816374269251065,
      0.3957185392967492,
      0.3571817867897743,
      0.41715561474532525,
      0.3320869831031704,
      0.43982232865571547,
      0.4379489245728095]}},
   10: {'Hard_Thr': {'((), {})': [0.3440405973843051,
      0.330535278377002,
      0.33822233364565907,
      0.3360254218756192,
      0.35587281765970386,
      0.33944258975556196,
      0.36001475912095643,
      0.35617436330521285,
      0.37176605005919794,
      0.35220059657081854]},
    'Soft_Thr': {'((), {})': [0.6365218164317165,
      0.6189570053461585,
      0.6617306759717

In [7]:
results_df = benchmark.get_results_as_df() # This formats the results on a DataFrame
results_df[1]

,Method,Parameter,Signal_id,Repetition,0,10,20
40,Hard_Thr,"((), {})",speech_1,0,1.142343,1.564916,2.723396
41,Hard_Thr,"((), {})",speech_1,1,1.119545,1.503485,2.566996
42,Hard_Thr,"((), {})",speech_1,2,1.127981,1.538451,2.690908
43,Hard_Thr,"((), {})",speech_1,3,1.132775,1.528458,2.684079
44,Hard_Thr,"((), {})",speech_1,4,1.148444,1.618737,2.785739
45,Hard_Thr,"((), {})",speech_1,5,1.128622,1.544002,2.712857
46,Hard_Thr,"((), {})",speech_1,6,1.142200,1.637577,2.893670
47,Hard_Thr,"((), {})",speech_1,7,1.135804,1.620108,2.923806
48,Hard_Thr,"((), {})",speech_1,8,1.162871,1.691029,3.021406
49,Hard_Thr,"((), {})",speech_1,9,1.140893,1.602033,2.900660


## Generating plots with the Results Interpreter.

In [8]:
# Summary interactive plots with Plotly
from plotly.offline import  iplot
interpreter = ResultsInterpreter(benchmark)
figs = interpreter.get_summary_plotlys(bars=True)
for fig in figs:
    fig.update_layout(yaxis_title="PESQ(x_est)/PESQ(x)")
    iplot(fig)


### Checking elapsed time for each method

In [9]:
df = interpreter.elapsed_time_summary()
df

,Mean,Std
"speech_1-Hard_Thr-((), {})",0.003018,0.000197
"speech_1-Soft_Thr-((), {})",0.003681,0.000264
"speech_2-Hard_Thr-((), {})",0.003497,0.000419
"speech_2-Soft_Thr-((), {})",0.003162,0.000792
